## Homework

In [82]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2021-03-31 12:06:55--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Распознаётся docs.google.com (docs.google.com)… 64.233.162.194, 2a00:1450:4010:c05::c2
Подключение к docs.google.com (docs.google.com)|64.233.162.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Moved Temporarily
Адрес: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ta045hgr1nunjkrah7a354g956fttfie/1617181575000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [переход]
Предупреждение: в HTTP шаблоны не поддерживаются.
--2021-03-31 12:06:58--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ta045hgr1nunjkrah7a354g956fttfie/1617181575000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Распознаётся doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)… 216.58.209.193, 2a00:1450:4026:803::2001
Подключение к 

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [1]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [3]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [4]:
X = df.drop(['price', 'id', 'timestamp'],1)
y = df['price']

In [5]:
X = X._get_numeric_data()

In [6]:
X.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,...,91,36,7,2,15,33,1,12,75,10
1,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,6.677245e+07,...,3,2,2,0,0,13,1,0,6,1
2,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,1.216448e+07,...,23,13,6,1,8,18,0,1,52,0
3,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.708040e+06,...,3,0,0,0,1,3,0,2,8,2
4,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,1.428699e+07,...,12,3,1,0,5,8,0,1,34,5


In [7]:
X = X.fillna(X.mode().iloc[0])

In [8]:
X.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,...,91,36,7,2,15,33,1,12,75,10
1,64,64.0,16.0,17.0,1.0,2014.0,2.0,1.0,2.0,6.677245e+07,...,3,2,2,0,0,13,1,0,6,1
2,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,1.216448e+07,...,23,13,6,1,8,18,0,1,52,0
3,71,49.0,2.0,17.0,1.0,2014.0,2.0,1.0,2.0,4.708040e+06,...,3,0,0,0,1,3,0,2,8,2
4,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,1.428699e+07,...,12,3,1,0,5,8,0,1,34,5


Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [134]:
X.shape

(19657, 279)

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 42)

Проверьте качество на отложенной выборке.

In [114]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
clf = DecisionTreeClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(mean_squared_error(y_test,y_pred,squared=False))

3092720.7704128544


In [120]:
from sklearn.model_selection import cross_val_score
a = (-(np.mean(cross_val_score(clf, X, y, cv=3, scoring = 'neg_mean_squared_error', n_jobs = -1))))
print(np.sqrt(a))

/home/pavel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


3166071.4427241473


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [163]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/home/pavel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [177]:

df = df.drop(df[(df['max_floor'] == 0) | (df['full_sq'] == 0)].index)

In [178]:
df['month'] = df['timestamp'].dt.month

In [179]:
df['f/f_max'] = df['floor']/df['max_floor']
df['k_sq/f_sq'] = df['kitch_sq']/df['full_sq']
df['month'] = df['timestamp'].dt.day

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [180]:
X = df.drop(['price', 'id', 'timestamp'],1)
y = df['price']

In [181]:
X = X._get_numeric_data()

In [182]:
X = X.fillna(X.mode().iloc[0])

In [183]:
X.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month,f/f_max,k_sq/f_sq
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,...,2,15,33,1,12,75,10,26,1.000000,1.000000
1,64,64.0,16.0,17.0,1.0,2014.0,2.0,1.0,2.0,6.677245e+07,...,0,0,13,1,0,6,1,4,1.000000,0.000000
2,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,1.216448e+07,...,1,8,18,0,1,52,0,5,0.529412,0.120482
3,71,49.0,2.0,17.0,1.0,2014.0,2.0,1.0,2.0,4.708040e+06,...,0,1,3,0,2,8,2,26,1.000000,0.000000
4,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,1.428699e+07,...,0,5,8,0,1,34,5,29,1.000000,0.100000


In [184]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 42)

In [186]:
clf = DecisionTreeClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(mean_squared_error(y_test,y_pred,squared=False))

5571968.830422425


### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [187]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
X = df.drop(['price', 'id', 'timestamp'],1)
y = df['price']
X = X._get_numeric_data()
X = X.fillna(X.mode().iloc[0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 42)


In [196]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(random_state=42,max_depth = 4)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(mean_squared_error(y_test,y_pred,squared=False))

3229384.956807871


In [197]:
a = (-(np.mean(cross_val_score(reg, X, y, cv=3, scoring = 'neg_mean_squared_error', n_jobs = -1))))
print(np.sqrt(a))

3210989.550525682


In [198]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
print(mean_squared_error(y_test,y_pred,squared=False))

6202523.064701841


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [220]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
df_invest = df.drop(df[(df['product_type'] =='Investment')].index)

In [221]:
df_own = df.drop(df[(df['product_type'] =='OwnerOccupier')].index)

In [222]:
X_invest = df_invest.drop(['price', 'id', 'timestamp'],1)
y_invest = df_invest['price']
X_invest = X_invest._get_numeric_data()
X_invest = X_invest.fillna(X.mode().iloc[0])

In [223]:
X_train_invest, X_test_invest, y_train_invest, y_test_invest = train_test_split(X_invest, y_invest, test_size = 0.8, random_state = 42)

In [224]:
reg = DecisionTreeRegressor(random_state=42,max_depth = 4)
reg.fit(X_train_invest, y_train_invest)
y_pred_invest = reg.predict(X_test_invest)
print(mean_squared_error(y_test_invest,y_pred_invest,squared=False))

2595079.2993084826


In [225]:
X_own = df_own.drop(['price', 'id', 'timestamp'],1)
y_own = df_own['price']
X_own = X_own._get_numeric_data()
X_own = X_own.fillna(X.mode().iloc[0])

In [226]:
X_train_own, X_test_own, y_train_own, y_test_own = train_test_split(X_own, y_own, test_size = 0.8, random_state = 42)

In [227]:
reg.fit(X_train_own, y_train_own)
y_pred_own = reg.predict(X_test_own)
print(mean_squared_error(y_test_own,y_pred_own,squared=False))

3493824.4122218667


### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [9]:
pip install catboost

     |████████████████████████████████| 67.3 MB 6.8 MB/s eta 0:00:013     |████████████████▎               | 34.2 MB 11.6 MB/s eta 0:00:03
     |████████████████████████████████| 13.2 MB 7.5 MB/s eta 0:00:01    |███████████████████████▍        | 9.6 MB 7.5 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=53bfe6b8cf73acbb1a1acd1cc7657de389051c560085f3c1c7b883337bd9c33f
  Stored in directory: /home/pavel/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [59]:
from catboost import  CatBoostRegressor

In [252]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
df_invest = df.drop(df[(df['product_type'] =='OwnerOccupier')].index)


In [253]:
X_invest = df_invest.drop(['price', 'id', 'timestamp'],1)
X_invest = X_invest.fillna(X_invest.mode().iloc[0])
y_invest = df_invest['price']
cat_columns = [
    'product_type',            #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
for i in cat_columns:
    X_invest[i] = X_invest[i].astype(str)

In [261]:
model = CatBoostRegressor(iterations=35, 
                          depth=5, 
                          learning_rate=0.6, 
                          loss_function='RMSE',cat_features = cat_columns)

In [262]:
from sklearn.model_selection import train_test_split

In [263]:
X_train_invest, X_test_invest, y_train_invest, y_test_invest = train_test_split(X_invest, y_invest, test_size = 0.8, random_state = 42)

In [264]:
model.fit(X_train_invest, y_train_invest)

0:	learn: 3706273.4717693	total: 8.65ms	remaining: 294ms
1:	learn: 3230699.4992212	total: 20.8ms	remaining: 343ms
2:	learn: 3050961.4686980	total: 29.1ms	remaining: 310ms
3:	learn: 2919849.6792875	total: 36.7ms	remaining: 285ms
4:	learn: 2838225.8627007	total: 44.3ms	remaining: 266ms
5:	learn: 2792728.4778473	total: 51.9ms	remaining: 251ms
6:	learn: 2761892.3676062	total: 59.2ms	remaining: 237ms
7:	learn: 2713058.1898711	total: 66.6ms	remaining: 225ms
8:	learn: 2660315.2367528	total: 74.7ms	remaining: 216ms
9:	learn: 2637724.4509354	total: 82.4ms	remaining: 206ms
10:	learn: 2608677.1515469	total: 90.1ms	remaining: 197ms
11:	learn: 2583176.1802900	total: 97.4ms	remaining: 187ms
12:	learn: 2574358.9172466	total: 105ms	remaining: 177ms
13:	learn: 2560694.9655440	total: 112ms	remaining: 168ms
14:	learn: 2533383.6114589	total: 119ms	remaining: 159ms
15:	learn: 2487775.5509778	total: 127ms	remaining: 150ms
16:	learn: 2478302.4367621	total: 134ms	remaining: 142ms
17:	learn: 2444577.0590217	to

In [265]:
y_pred = model.predict(X_test_invest)

In [266]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_pred,y_test_invest,squared=False))

3460444.592864955


In [267]:
from sklearn.model_selection import cross_val_score
a = (-(np.mean(cross_val_score(model, X_invest, y_invest, cv=3, scoring = 'neg_mean_squared_error', n_jobs = -1))))
print(np.sqrt(a))

3176251.4602305074


In [191]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
X = df.drop(['price', 'id', 'timestamp'],1)
y = df['price']
X = X._get_numeric_data()
X = X.fillna(X.mode().iloc[0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 42)

In [249]:
model = CatBoostRegressor(iterations=25, 
                          depth=5, 
                          learning_rate=0.5, 
                          loss_function='RMSE',)

In [250]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(mean_squared_error(y_pred,y_test,squared=False))

0:	learn: 3768304.3826268	total: 12.8ms	remaining: 306ms
1:	learn: 3269120.4381110	total: 23.9ms	remaining: 275ms
2:	learn: 3001389.3363773	total: 32.7ms	remaining: 240ms
3:	learn: 2811332.7781049	total: 40.7ms	remaining: 214ms
4:	learn: 2759084.8728346	total: 48.6ms	remaining: 194ms
5:	learn: 2672879.9276719	total: 55.9ms	remaining: 177ms
6:	learn: 2602664.1744047	total: 63.1ms	remaining: 162ms
7:	learn: 2521423.4774203	total: 70.3ms	remaining: 149ms
8:	learn: 2493563.1208568	total: 77.3ms	remaining: 137ms
9:	learn: 2455377.0555989	total: 84.5ms	remaining: 127ms
10:	learn: 2413720.4505378	total: 91.8ms	remaining: 117ms
11:	learn: 2394581.3410222	total: 98.8ms	remaining: 107ms
12:	learn: 2351625.3082299	total: 106ms	remaining: 97.9ms
13:	learn: 2327790.4300801	total: 113ms	remaining: 89ms
14:	learn: 2318981.3643947	total: 120ms	remaining: 80.1ms
15:	learn: 2291140.3553342	total: 127ms	remaining: 71.5ms
16:	learn: 2270756.3094535	total: 135ms	remaining: 63.5ms
17:	learn: 2240700.1313680

In [251]:
a = (-(np.mean(cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_squared_error', n_jobs = -1))))
print(np.sqrt(a))

2759256.235952743
